## Install modules

In [4]:
!pip install praw
!pip3 install newspaper3k

## Import functions and grabbing reddit data

In [5]:
import praw
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from newspaper import Article
import nltk
nltk.download('punkt')
# Reddit credentials, password stored in .env 
# PRAW setup goes here

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
# here is a read only reddit scraper object. This can be used to scan and scrape stories.
# the parameters below must be provided for the scraper to work.
# This is what will be used to pull stories from the reddit's online forum.
reddit = praw.Reddit(client_id='D1NQHCUDKQg3TA',
                     client_secret='GJn8WpYkFXi5CREeEdwKCiRZz4w',
                     user_agent="windows:reddit:v1(david_cruz_0202)",
                     username="david_cruz_0202")

# The reddit API doesn't require developer credentials for a read only API
# which is what we're doing. So there are no passwords being held.

In [7]:
# Grabbing 100 hottest posts on Reddit at the moment. Will filter for police use of force later

data = []

# other possible subreddits: publicfreakout, allcopnodonut
for submission in reddit.subreddit("news").hot(limit=100):
  data.append([submission.id, submission.title, submission.url])

# We'll need a way to get coordinates for a given post, before we include that in df
col_names = ['id', 'title', 'url']
df = pd.DataFrame(data, columns=col_names)

df.head()

,id,title,url
0,jb2ujn,Bronze statue of Ruth Bader Ginsburg to be unv...,https://abcnews.go.com/US/bronze-statue-ruth-b...
1,jb1llg,Amy Cooper Made 2nd 911 Call to Falsely Accuse...,https://www.nytimes.com/2020/10/14/nyregion/am...
2,jayojh,Mexico identifies two women who may have recei...,https://edition.cnn.com/2020/10/13/world/mexic...
3,jb0ssy,YouTube bans coronavirus vaccine misinformation,https://www.reuters.com/article/us-health-coro...
4,jb3b4e,White Michigan man accused of attacking Black ...,https://www.nbcnews.com/news/us-news/white-mic...


## Deprecated tag based filtering system

In [8]:
# set up some sort of corpus of keywords to snag specific reddit entries
all_locs = pd.read_json('https://raw.githubusercontent.com/2020PB/police-brutality/data_build/all-locations-v2.json')
all_locs = pd.json_normalize(all_locs['data'])
all_locs = all_locs.drop(columns=['edit_at','id'])
def cleanlinks(json):
    links_out = []
    for link in json:
        links_out.append(link['url'])
    return links_out
all_locs['links'] = all_locs['links'].apply(cleanlinks)
all_locs['date'] = pd.to_datetime(all_locs['date'],format='%Y-%m-%d')
all_tags = all_locs['tags'].copy()
tags = set()
for taglist in all_tags:
  for tag in taglist:
    if tag not in tags:
      tags.add(tag)
# manually remove needless tags
print(tags)
tags.discard('')
tags.discard('medic')
tags.discard('bike')
tags.discard('non-protest')
tags.discard('pregnant')
tags.discard('lgbtq+')
tags.discard('racial-profiling')
tags.discard('legal-observer')
tags.discard('tear-gas-canister')
tags.discard('politician')
tags.discard('incitement')
tags.discard('homeless')
tags.discard('shoot')
tags.discard('strike')
tags.discard('elderly')
tags.discard('vehicle')
tags.discard('inhumane-treatment')
tags.discard('journalist')
tags.discard('throw')
tags.discard('explosive')
tags.discard('threaten')
tags.discard('horse')
tags.discard('shove')
tags.discard('child')
tags.discard('shield')
tags.discard('dog')
tags.discard('knee')
tags.discard('protester')
tags.discard('gun')
tags.discard('conceal')
tags.discard('bystander')
tags.discard('grab')
tags.discard('push')
tags.discard('zip-tie')
tags.discard('spray')
tags.discard('drive')
tags.discard('person-with-disability')
tags.discard('celebrity')
tags.discard('projectile')
tags.discard('beat')
newtags = set()
for tag in tags:
  tag = newtags.add(tag.replace('-',' '))
tags = newtags
print(tags)

{'', 'pregnant', 'journalist', 'inhumane-treatment', 'marking-round', 'choke', 'drive', 'foam-bullet', 'tear-gas-canister', 'shoot', 'live-round', 'knee', 'hide-badge', 'medic', 'threaten', 'celebrity', 'homeless', 'arrest', 'knee-on-neck', 'lrad', 'shove', 'sexual-assault', 'taser', 'shield', 'pepper-spray', 'elderly', 'abuse-of-power', 'gas', 'tase', 'push', 'racial-profiling', 'beat', 'spray', 'pepper-ball', 'non-protest', 'conceal', 'grab', 'kick', 'property-destruction', 'zip-tie', 'rubber-bullet', 'bike', 'lgbtq+', 'dog', 'politician', 'punch', 'bystander', 'tackle', 'headlock', 'tear-gas', 'strike', 'person-with-disability', 'bean-bag', 'mace', 'baton', 'incitement', 'child', 'throw', 'less-lethal', 'explosive', 'horse', 'vehicle', 'legal-observer', 'body-cam', 'death', 'gun', 'stun-grenade', 'projectile', 'protester', 'wooden-bullet'}
{'wooden bullet', 'live round', 'hide badge', 'choke', 'abuse of power', 'body cam', 'arrest', 'lrad', 'rubber bullet', 'taser', 'pepper spray', 

In [9]:
# get the url of the reddit post
for url in df['url']:
  # get the HTML from the url
  try:
    r = requests.get(url, timeout=10)
  except:
    continue
  soup = BeautifulSoup(r.text)
  # get tags from metadata for the site
  sitetags = set()
  for meta in soup.find_all('meta'):
    if meta is None:
      continue
    meta = str(meta)
    meta = meta.lower()
    meta = re.sub('[\W_]+',' ', meta)
    for tag in str(meta).split():
      sitetags.add(tag)
  tags_final = sitetags & tags
  if tags_final:
    # some matches found, print title and matched tags
    print(soup.title.text.strip(), tags_final)

Gretchen Whitmer: Sheriff spoke in defense of accused domestic terrorists - CNN {'arrest'}
Dutch woman dies after catching COVID-19 twice, the first reported reinfection death | CTV News {'death'}
Fairview mayor resigns following Target shoplifting arrest {'arrest'}
Mario Molina, 77, Dies; Sounded an Alarm on the Ozone Layer - The New York Times {'gas'}
Tennessee man charged in beating death of 2-year-old boy {'death'}
Researchers find elevated radiation near U.S. fracking sites | Reuters {'gas'}
Conchata Ferrell Dead: ‘Two and a Half Men’ Co-Star, ‘L.A. Law Was 77 – Deadline {'arrest'}
West Virginia babysitter convicted in 2018 death of toddler {'death'}
Dutch government backs euthanasia for under-12s | World news | The Guardian {'death'}
Philippines: Anger over death of baby separated from jailed mother - BBC News {'death'}


## Old `requests` based text grabbing function

In [10]:
# Test grabbing the contents of the articles themselves to imporve spaCy NLP
# get the url of the reddit post
content_list = []
df_snip = df['url']
for id_url in df_snip:
  # get the HTML from the url
  try:
    r = requests.get(id_url, timeout=10)
  except:
    content_list.append('')
    continue
  soup = BeautifulSoup(r.text)
  # get text from website
  output_text = " ".join([x.text for x in soup.find_all('p')])
  output_text = re.sub("[^a-zA-Z0-9.,']+", ' ', output_text).strip()
  content_list.append(output_text)

print(df.shape, len(content_list))
df['text'] = content_list
df = df[df['text'] != '']
df.head()

(100, 3) 100


,id,title,url,text
0,jb2ujn,Bronze statue of Ruth Bader Ginsburg to be unv...,https://abcnews.go.com/US/bronze-statue-ruth-b...,A bronze statue of hometown legend Supreme Cou...
1,jb1llg,Amy Cooper Made 2nd 911 Call to Falsely Accuse...,https://www.nytimes.com/2020/10/14/nyregion/am...,Advertisement Supported by Prosecutors said Ms...
2,jayojh,Mexico identifies two women who may have recei...,https://edition.cnn.com/2020/10/13/world/mexic...,"By Natalie Gall n, CNN Updated 0155 GMT 0955 H..."
3,jb0ssy,YouTube bans coronavirus vaccine misinformation,https://www.reuters.com/article/us-health-coro...,Discover Thomson Reuters By Elizabeth Cullifor...
4,jb3b4e,White Michigan man accused of attacking Black ...,https://www.nbcnews.com/news/us-news/white-mic...,Sections TV Featured More from NBC Follow NBC ...


## New `newspaper3k` based text extraction system

In [11]:
# set up future columns
content_list = []
date_list = []
tokens_list = []
df_snip = df['url']
# go through each URL and use newspaper3k to extract data
for id_url in df_snip:
  # use newspaper3k to extract text
  article = Article(id_url)
  article.download()
  # if the article doesn't download, the error is thrown in parse()
  try:
    article.parse()
  except:
    # add null values to show no connection
    content_list.append(None)
    date_list.append(None)
    continue
  content_list.append(article.text)
  # this will be null if newspaper3k can't find it
  date_list.append(article.publish_date)
df['text'] = content_list
df['date'] = date_list

In [12]:
# show results
print(df.shape)
df.head()

(98, 5)


,id,title,url,text,date
0,jb2ujn,Bronze statue of Ruth Bader Ginsburg to be unv...,https://abcnews.go.com/US/bronze-statue-ruth-b...,Bronze statue of Ruth Bader Ginsburg to be unv...,None
1,jb1llg,Amy Cooper Made 2nd 911 Call to Falsely Accuse...,https://www.nytimes.com/2020/10/14/nyregion/am...,"Amy Cooper, the white woman who called the pol...",2020-10-14 00:00:00
2,jayojh,Mexico identifies two women who may have recei...,https://edition.cnn.com/2020/10/13/world/mexic...,Mexico City (CNN) Mexico has identified two wo...,2020-10-13 00:00:00
3,jb0ssy,YouTube bans coronavirus vaccine misinformation,https://www.reuters.com/article/us-health-coro...,(Reuters) - Alphabet Inc’s YouTube said on Wed...,2020-10-14 13:52:25+00:00
4,jb3b4e,White Michigan man accused of attacking Black ...,https://www.nbcnews.com/news/us-news/white-mic...,A white man in Michigan faces a federal hate c...,2020-10-14 17:42:38+00:00


In [13]:
# show losses
df.isnull().sum()

id        0
title     0
url       0
text      1
date     36
dtype: int64

In [14]:
# Remove all entries with missing data
df = df.dropna()
print(df.shape)
df = df.reset_index()
df = df.drop(columns='index')
df.head()

(62, 5)


,id,title,url,text,date
0,jb1llg,Amy Cooper Made 2nd 911 Call to Falsely Accuse...,https://www.nytimes.com/2020/10/14/nyregion/am...,"Amy Cooper, the white woman who called the pol...",2020-10-14 00:00:00
1,jayojh,Mexico identifies two women who may have recei...,https://edition.cnn.com/2020/10/13/world/mexic...,Mexico City (CNN) Mexico has identified two wo...,2020-10-13 00:00:00
2,jb0ssy,YouTube bans coronavirus vaccine misinformation,https://www.reuters.com/article/us-health-coro...,(Reuters) - Alphabet Inc’s YouTube said on Wed...,2020-10-14 13:52:25+00:00
3,jb3b4e,White Michigan man accused of attacking Black ...,https://www.nbcnews.com/news/us-news/white-mic...,A white man in Michigan faces a federal hate c...,2020-10-14 17:42:38+00:00
4,jazda1,Sheriff spoke in defense of accused domestic t...,https://www.cnn.com/2020/10/14/us/michigan-she...,"Grand Rapids, Michigan (CNN) On May 18, the FB...",2020-10-14 00:00:00


## Gather location information

In [16]:
locs_df = pd.read_csv('cities_states.csv')
def lowerify(text):
  return text.lower()
locs_df = locs_df.drop(columns=['Unnamed: 0','country'])
locs_df['city_ascii'] = locs_df['city_ascii'].apply(lowerify)
locs_df['admin_name'] = locs_df['admin_name'].apply(lowerify)
locs_df.head()

,city_ascii,admin_name,lat,lng
0,new york,new york,40.6943,-73.9249
1,los angeles,california,34.1139,-118.4068
2,chicago,illinois,41.8373,-87.6862
3,miami,florida,25.7839,-80.2102
4,dallas,texas,32.7936,-96.7662


## Old `collections` based approach

In [17]:
from collections import Counter
import re


# get list of states
states_list = list(locs_df.admin_name.unique())
states_map = {}
# for each state, map their respective cities
for state in states_list:
  states_map[state] = locs_df[locs_df['admin_name'] == state]['city_ascii'].to_list()

# get a list of tokens from the text
test_tokens = re.sub('[\W]+',' ',df['text'][3]).lower().split()

# put the tokens into a Counter
c = Counter(test_tokens)

# Check, for each state, which ones come back with a value of more than one
state_counts = {}
for state in states_list:
  if c[state] > 0:
    state_counts[state] = c[state]

print(test_tokens[:20])
print(state_counts)

city_max = max(state_counts, key=state_counts.get)
city_counts = {}
for city in states_map[city_max]:
  if c[city] > 0:
    city_counts[city] = c[city]

print(states_map[city_max])
print(city_counts)

['a', 'white', 'man', 'in', 'michigan', 'faces', 'a', 'federal', 'hate', 'crime', 'charge', 'for', 'allegedly', 'attacking', 'a', 'black', 'teenager', 'with', 'a', 'bike']
{'michigan': 2}
['detroit', 'grand rapids', 'flint', 'ann arbor', 'lansing', 'kalamazoo', 'muskegon', 'warren', 'sterling heights', 'south lyon', 'saginaw', 'holland', 'lincoln park', 'eastpointe', 'oak park', 'hamtramck', 'hazel park', 'berkley', 'harper woods', 'clawson', 'dearborn', 'livonia', 'jackson', 'port huron', 'troy', 'westland', 'farmington hills', 'battle creek', 'wyoming', 'rochester hills', 'southfield', 'bay city', 'taylor', 'benton harbor', 'pontiac', 'novi', 'st. clair shores', 'midland', 'royal oak', 'dearborn heights', 'kentwood', 'traverse city', 'monroe', 'east lansing', 'portage', 'roseville', 'adrian', 'mount pleasant', 'madison heights', 'southgate', 'burton', 'forest hills', 'allen park', 'garden city', 'marquette', 'wyandotte', 'holt', 'walker', 'waverly', 'norton shores', 'inkster', 'romul

## New `spacy` based location extraction method

In [18]:
import spacy


# prep spacy
nlp = spacy.load('en_core_web_sm')
# for each article, perform NLP on its text
tokens_list = []
for text in df['text']:
  doc = nlp(text)

  ents = [(e.text, e.label_) for e in doc.ents if e.label_ == 'GPE']
  tokens_list.append(ents)

df['tokens'] = tokens_list

In [20]:
# what's the results?
df['tokens']

0                                    [(Manhattan, GPE)]
1     [(Mexico City, GPE), (Mexico, GPE), (Ocilla, G...
2                                                    []
3     [(Michigan, GPE), (Michigan, GPE), (Mouat, GPE...
4     [(Grand Rapids, GPE), (Michigan, GPE), (Michig...
                            ...                        
57                                                   []
58    [(Arizona, GPE), (Vietnam, GPE), (Washington, ...
59    [(Baikonur, GPE), (Kazakhstan, GPE), (Russia, ...
60    [(Armenia, GPE), (Azerbaijan, GPE), (Turkey, G...
61    [(Kingston, GPE), (Kingston, GPE), (City, GPE)...
Name: tokens, Length: 62, dtype: object

In [21]:
df.dropna()

,id,title,url,text,date,tokens
0,jb1llg,Amy Cooper Made 2nd 911 Call to Falsely Accuse...,https://www.nytimes.com/2020/10/14/nyregion/am...,"Amy Cooper, the white woman who called the pol...",2020-10-14 00:00:00,"[(Manhattan, GPE)]"
1,jayojh,Mexico identifies two women who may have recei...,https://edition.cnn.com/2020/10/13/world/mexic...,Mexico City (CNN) Mexico has identified two wo...,2020-10-13 00:00:00,"[(Mexico City, GPE), (Mexico, GPE), (Ocilla, G..."
2,jb0ssy,YouTube bans coronavirus vaccine misinformation,https://www.reuters.com/article/us-health-coro...,(Reuters) - Alphabet Inc’s YouTube said on Wed...,2020-10-14 13:52:25+00:00,[]
3,jb3b4e,White Michigan man accused of attacking Black ...,https://www.nbcnews.com/news/us-news/white-mic...,A white man in Michigan faces a federal hate c...,2020-10-14 17:42:38+00:00,"[(Michigan, GPE), (Michigan, GPE), (Mouat, GPE..."
4,jazda1,Sheriff spoke in defense of accused domestic t...,https://www.cnn.com/2020/10/14/us/michigan-she...,"Grand Rapids, Michigan (CNN) On May 18, the FB...",2020-10-14 00:00:00,"[(Grand Rapids, GPE), (Michigan, GPE), (Michig..."
...,...,...,...,...,...,...
57,ja41do,Johnson & Johnson pauses Covid-19 vaccine tria...,https://www.cnn.com/2020/10/12/health/johnson-...,(CNN) Drugmaker Johnson & Johnson said Monday ...,2020-10-12 00:00:00,[]
58,ja98vg,"Roberta McCain, Mother of Sen. John McCain, Di...",https://www.nytimes.com/2020/10/12/us/roberta-...,"Roberta McCain, whose son, Senator John McCain...",2020-10-12 00:00:00,"[(Arizona, GPE), (Vietnam, GPE), (Washington, ..."
59,jaywjo,Russian-US crew launches on fast track to the ...,https://apnews.com/article/moscow-kazakhstan-r...,In this image made from video footage released...,2020-10-14 05:52:34+00:00,"[(Baikonur, GPE), (Kazakhstan, GPE), (Russia, ..."
60,javxi0,Armenia Supporters Block Traffic in Various Lo...,https://www.nbclosangeles.com/news/local/appar...,Demonstrators in support of Armenia in its con...,2020-10-13 20:48:07+00:00,"[(Armenia, GPE), (Azerbaijan, GPE), (Turkey, G..."
